In [1]:
import aesara 
import aesara.tensor as at
import arviz as az
import matplotlib.pyplot as plt
import numpy as np 
import pymc as pm 
import xarray as xr 
import pandas as pd 
import pymc.sampling_jax
import numpyro 

print(f"Running on PyMC v{pm.__version__}")

Running on PyMC v4.1.3


/Users/conor/miniconda3/envs/pymc_non_dev/lib/python3.10/site-packages/pymc/sampling_jax.py:36: UserWarning: This module is experimental.
  warnings.warn("This module is experimental.")


#### Data: loading in the Scottish lip cancer data

In [2]:
df_scot_cancer = pd.read_csv(pm.get_data("scotland_lips_cancer.csv"))

# observed cancer counts
y = df_scot_cancer["CANCER"].values

# number of observations
N = len(y)

# expected cancer counts E for each county: this is calculated using age-standardized rates of the local population
E = df_scot_cancer["CEXP"].values
logE = np.log(E)

# proportion of the population engaged in agriculture, forestry, or fishing
x = df_scot_cancer["AFF"].values / 10.0

# spatial adjacency information: column `ADJ` contains list entries which are preprocessed to obtain adj as a list of lists
adj = (
    df_scot_cancer["ADJ"].apply(lambda x: [int(val) for val in x.strip("][").split(",")]).to_list()
)

# change to Python indexing (i.e. -1)
for i in range(len(adj)):
    for j in range(len(adj[i])):
        adj[i][j] = adj[i][j] - 1

# storing the adjacency matrix as a two-dimensional np.array
adj_matrix = np.zeros((N, N), dtype="int32")

for area in range(N):
    adj_matrix[area, adj[area]] = 1

In [3]:
# getting the necessary information to use for a spatial model specification 
N_edges = (adj_matrix == 1).sum()
node1 = np.where(adj_matrix == 1)[0] 
node2 = np.where(adj_matrix == 1)[1] 

In [4]:
coords = {"num_areas": N, 
          "num_neighbours": N_edges}

# Defining relevant functions for an ICAR prior

Writing a term proportional to the $\log p$ of the ICAR component, in terms of pairwise differences you get
\begin{align}
\log p(\boldsymbol{\phi})\propto \frac{-1}{2}\sum_{j\sim i}(\phi_i - \phi_j)^2.
\end{align}
Note that this is for the instance where $\tau=1$, and so below we experiment with including the scale parameter within the `pm.Potential` call. 

In [5]:
# creating the pairwise specification for the ICAR prior 
def icar_pairwise_diff(phi, node1, node2):
    return -0.5*((phi[node1]-phi[node2])**2).sum()
# pairwise specification of the ICAR prior, with the inclusion of the scale parameter

# BYM Model

The BYM model that we will now fit to the cancer data is written as 

\begin{align}
y_i &\sim \text{Poisson}(\mu_i)\\
\log \mu_i &= \beta_0 + \beta_1 x_i + \log E_i + \theta_i + \phi_i \\
\boldsymbol{\theta} &\sim \text{Normal}\big (0, \frac{1}{\tau_{\theta}^2}\big ) \\
\boldsymbol{\phi} &\sim \text{Normal}\big (0, \big (\tau_\phi^2\big[\mathbf{D}-\mathbf{W} \big]\big )^{-1}\big )\\
\tau_\theta^2 &\sim \text{Gamma}(3.2761, 1.81)\\
\tau_\phi^2 &\sim \text{Gamma}(1, 1)\\
\beta_0, \beta_1 &\sim \text{Normal}(0, 5).
\end{align}

The random effect for each of the area's $i$ can be considered as 
\begin{align}
\psi_i = \theta_i + \phi_i,  
\end{align}
where $\phi_i$ is the spatial random effect modelled using an ICAR prior, and $\theta_i$ is an independent random effect, that accounts for the additional variation in the data set, that can neither be explained by spatial dependencies or variatibility from the Poisson distribution. 

To ensure identifiability, we need the vectors $\boldsymbol{\phi}$ and $\boldsymbol{\theta}$ to independently sum to $0$. We will try and write the BYM model below through two different parameterizations. 

In [8]:
with pm.Model(coords={"num_areas": np.arange(N)}) as bym_model:
    # precision priors 
    tau_theta = pm.Gamma("tau_theta", alpha=3.2761, beta=1.81)
    tau_phi = pm.Gamma("tau_phi", alpha=1, beta=1)

    # transform to standard deviation 
    sigma_phi = pm.Deterministic("sigma_phi", 1/at.sqrt(tau_phi))

    # independent random effect prior
    theta = pm.Normal("theta", mu=0, sigma=1/at.sqrt(tau_theta), dims="num_areas")
    # spatial ICAR random effect prior 
    phi = pm.Flat("phi", dims="num_areas")
    pm.Potential("spatial_diff", icar_pairwise_diff(phi, node1, node2)) 

    # constraints for each of the random effect vectors 
    zero_constraint = pm.Normal.dist(mu=0.0, sigma=np.sqrt(0.001))
    pm.Potential("zero_sum_theta", pm.logp(zero_constraint, pm.math.sum(theta))) 
    pm.Potential("zero_sum_phi", pm.logp(zero_constraint, pm.math.sum(phi)))

    # regression coefficient priors 
    beta0 = pm.Normal("beta0", mu=0, sigma=5)
    beta1 = pm.Normal("beta1", mu=0, sigma=5)
    
    # linear predictor 
    eta = pm.Deterministic("eta", logE + beta0 + beta1*x + sigma_phi*phi + theta, dims="num_areas") 

    # likelihood
    obs = pm.Poisson("obs", at.exp(eta), observed=y, dims="num_areas")

In [9]:
with bym_model:
    idata = pm.sample()

Auto-assigning NUTS sampler...
INFO:pymc:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau_theta, tau_phi, theta, phi, beta0, beta1]
INFO:pymc:NUTS: [tau_theta, tau_phi, theta, phi, beta0, beta1]


/Users/conor/miniconda3/envs/pymc_non_dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/conor/miniconda3/envs/pymc_non_dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/conor/miniconda3/envs/pymc_non_dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/conor/miniconda3/envs/pymc_non_dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 197 seconds.
INFO:pymc:Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 197 seconds.
